In [1]:
import torch
from torch.nn import Linear
import torch.nn.functional as F

from torch_geometric.nn import GCNConv, SAGEConv
from torch_geometric.data import Data, DataLoader

print(torch.__version__)
print(torch.cuda.is_available()) 
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(torch.cuda.current_device()))

2.8.0+cu128
True
1
NVIDIA GeForce RTX 4090


https://arxiv.org/pdf/2303.16741
Who you Play Affects How you Play: Predicting Sports Performance using Grpah Attention with Temporal Convolution 

https://arxiv.org/abs/2207.13191
GCN-WP -- Semi-Supervised Graph Convolutional Networks for Win Prediction in Esports

https://www.firecrawl.dev/?twclid=2-270mk2awl7ihu5h3nss6shp14
Turn websites into AI LLM ready data 

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder


In [3]:
df_history = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\model_results_train_v2.csv')
df_results = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\model_results_v2.csv')


In [4]:
def build_fighter_ids(df, red_col, blue_col):
    # Combine all fighters and get unique names
    df = df.copy()
    all_fighters = pd.concat([df[red_col], df[blue_col]]).unique()
    
    # Create mapping from fighter name to unique ID
    fighter_to_id = {fighter: idx for idx, fighter in enumerate(all_fighters)}
    
    # Map IDs to red and blue columns
    df['id_red'] = df[red_col].map(fighter_to_id)
    df['id_blue'] = df[blue_col].map(fighter_to_id)
    
    return df

def build_train_test(df, cols):
    # Filter out draws and drop rows with missing values
    df = df[df['winner'] != 2]
    df = df[cols].dropna().copy()
    df = df.sort_values(by='date', ascending=True).reset_index(drop=True)

    # Split index for train/test
    train_len = int(df.shape[0] * 0.85)
    
    # Add split flag
    df['split'] = ['train' if i < train_len else 'test' for i in range(df.shape[0])]

    # Keep identifiers for later
    id_cols = ['red_fighter', 'blue_fighter', 'date']

    # Separate features and target
    y = df['winner']

    X = df.drop(columns=['winner'])

    # Encode categorical columns
    le = LabelEncoder()
    if 'weight_class' in X.columns:
        X['weight_class'] = le.fit_transform(X['weight_class'])

    # Identify categorical and numerical columns (excluding id columns)
    cat_cols = [c for c in X.select_dtypes(include=['object', 'category']).columns if c not in id_cols + ['split']]
    numerical_cols = [c for c in X.columns if c not in cat_cols + id_cols + ['split']]

    # Scale numerical columns
    scaler = StandardScaler()
    X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

    # Combine features + id columns + target + split into one DataFrame
    combined_df = X.copy()
    combined_df['winner'] = y.values

    # Reorder columns: id_cols first, then features, then winner, then split
    combined_df = combined_df[id_cols + [c for c in X.columns if c not in id_cols + ['split']] + ['winner', 'split']]

    return combined_df

def build_temporal_graph(df, red_id_col, blue_id_col, date_col,
                         node_cols_red, node_cols_blue,
                         split_col='split'):

    df = df.sort_values(by=date_col, ascending=True).reset_index(drop=True)
    df = build_fighter_ids(df, 'red_fighter', 'blue_fighter')

    red_ids = df[red_id_col].values
    blue_ids = df[blue_id_col].values
    dates = pd.to_datetime(df[date_col])
    splits = df[split_col].values  # 'train' or 'test' for each fight

    # Build edge_index (flattened)
    edge_index = torch.tensor([
        [x for pair in zip(red_ids, blue_ids) for x in pair],  # red→blue
        [x for pair in zip(blue_ids, red_ids) for x in pair]   # blue→red
    ], dtype=torch.long)

    # Build date_index aligned with edge_index
    date_index = np.array([d for d in dates for _ in (0,1)])
    split_index = np.array([s for s in splits for _ in (0,1)])  # duplicate for both directions
    num_edges = edge_index.shape[1]

    node_features_source = []
    node_features_target = []
    edge_features_list = []

    for i in range(num_edges):
        src_node = edge_index[0, i].item()
        tgt_node = edge_index[1, i].item()
        edge_date = date_index[i]

        # Determine direction (red->blue or blue->red)
        if i % 2 == 0: 
            mask = (
                (df[red_id_col] == src_node) &
                (df[blue_id_col] == tgt_node) &
                (pd.to_datetime(df[date_col]) == edge_date)
            )
            row = df.loc[mask].iloc[0]
            node_features_source.append(torch.tensor(row[node_cols_red].values.astype(float)))
            node_features_target.append(torch.tensor(row[node_cols_blue].values.astype(float)))
        else: 
            mask = (
                (df[blue_id_col] == src_node) &
                (df[red_id_col] == tgt_node) &
                (pd.to_datetime(df[date_col]) == edge_date)
            )
            row = df.loc[mask].iloc[0]
            node_features_source.append(torch.tensor(row[node_cols_blue].values.astype(float)))
            node_features_target.append(torch.tensor(row[node_cols_red].values.astype(float)))

        # Edge label
        if row['winner'] == 1:
            edge_features_list.append(torch.tensor([1 if i % 2 == 0 else 0]))
        else:
            edge_features_list.append(torch.tensor([0 if i % 2 == 0 else 1]))

    # Stack features
    node_features_source = torch.stack(node_features_source)
    node_features_target = torch.stack(node_features_target)
    edge_features_tensor = torch.stack(edge_features_list)

    # Unique nodes and mapping
    all_nodes = torch.unique(edge_index)
    node_id_to_idx = {node.item(): i for i, node in enumerate(all_nodes)}
    num_nodes = len(all_nodes)
    node_feat_dim = node_features_source.shape[1]

    node_features = torch.zeros((num_nodes, node_feat_dim), dtype=torch.float)
    for i in range(edge_index.shape[1]):
        src_idx = node_id_to_idx[edge_index[0, i].item()]
        tgt_idx = node_id_to_idx[edge_index[1, i].item()]
        node_features[src_idx] = node_features_source[i]
        node_features[tgt_idx] = node_features_target[i]

    edge_index_mapped = torch.tensor([
        [node_id_to_idx[n.item()] for n in edge_index[0]],
        [node_id_to_idx[n.item()] for n in edge_index[1]]
    ], dtype=torch.long)

    # Create train/test masks for edges
    train_mask = torch.tensor([s == 'train' for s in split_index], dtype=torch.bool)
    test_mask = torch.tensor([s == 'test' for s in split_index], dtype=torch.bool)

    # Build PyG Data object
    data = Data(
        x=node_features,
        edge_index=edge_index_mapped,
        
        y=edge_features_tensor,
        train_mask=train_mask,
        test_mask=test_mask
    )
    #edge_attr=edge_features_tensor,
    return data


In [5]:
df_v2 = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\entire_odds_stats_v2.csv')
model_cols_red = ['reach_red', 'red_age', 'height_red', 'total_bonus_red', 
 'leg_strikes_pm_red', 'body_strikes_pm_red', 'head_strikes_pm_red',
 'clinch_strikes_pm_red', 'ground_strikes_pm_red', 'control_pr_red',
 'kd_pr_red', 'reverse_pr_red', 'sigstrikes_absorbed_pm_red', 
 'sigstrikes_pm_red', 'sub_att_pr_red', 'red_td_accuracy_avg',
 'red_td_defense_avg', 'red_td_landed_total', 'red_td_defended_total', 
 'ratio_td_red', 'red_sigstrike_accuracy_avg', 'red_sigstrike_defense_avg', 
 'red_kd_total', 'red_elo', 'red_glicko', 'red_glicko_rd', 'math_red', 
 'months_since_red', 'red_win_streak', 'red_lose_streak', 'win_pct_red',
 'num_fights_red', 'num_wins_red', 'num_losses_red', 'decision_wins_red', 
 'ko_wins_red', 'sub_wins_red', 'close1_red', 'close2_red', 'open_red']

model_cols_blue = ['reach_blue', 'blue_age', 'height_blue', 'total_bonus_blue', 
 'leg_strikes_pm_blue', 'body_strikes_pm_blue', 'head_strikes_pm_blue',
 'clinch_strikes_pm_blue', 'ground_strikes_pm_blue', 'control_pr_blue',
 'kd_pr_blue', 'reverse_pr_blue', 'sigstrikes_absorbed_pm_blue', 
 'sigstrikes_pm_blue', 'sub_att_pr_blue', 'blue_td_accuracy_avg',
 'blue_td_defense_avg', 'blue_td_landed_total', 'blue_td_defended_total', 
 'ratio_td_blue', 'blue_sigstrike_accuracy_avg', 'blue_sigstrike_defense_avg', 
 'blue_kd_total', 'blue_elo', 'blue_glicko', 'blue_glicko_rd', 'math_blue', 
 'months_since_blue', 'blue_win_streak', 'blue_lose_streak', 'win_pct_blue',
 'num_fights_blue', 'num_wins_blue', 'num_losses_blue', 'decision_wins_blue', 
 'ko_wins_blue', 'sub_wins_blue', 'close1_blue', 'close2_blue', 'open_blue']

graph_cols = ['red_fighter','blue_fighter','date', 'winner']

all_cols = model_cols_red + model_cols_blue + graph_cols

In [6]:
df_graph = build_train_test(df_v2, all_cols)
df_graph.head()

,red_fighter,blue_fighter,date,reach_red,red_age,height_red,total_bonus_red,leg_strikes_pm_red,body_strikes_pm_red,head_strikes_pm_red,...,num_wins_blue,num_losses_blue,decision_wins_blue,ko_wins_blue,sub_wins_blue,close1_blue,close2_blue,open_blue,winner,split
0,sean sherk,hermes franca,2007-07-07,-1.177550,2.380351,-1.170391,-0.227361,-0.568795,-1.061221,-0.759626,...,0.296671,-0.150864,-0.366751,0.331886,0.864428,0.737695,0.634364,0.803116,1,train
1,anderson silva,nate marquardt,2007-07-07,1.122395,2.022037,1.059753,-0.729220,-0.660576,2.428095,-0.325983,...,0.020272,-1.057681,0.658227,-0.758923,0.135477,0.352965,0.242210,0.563170,1,train
2,frank mir,antoni hardonk,2007-08-25,1.582385,1.305407,1.338521,-0.729220,-0.851785,-0.339825,-1.540264,...,-0.808928,-0.604273,-0.879240,-0.213519,-0.593475,0.429911,0.320641,-0.732536,1,train
3,patrick cote,kendall grove,2007-08-25,0.662406,1.126250,0.223449,-0.729220,-0.412004,-0.908303,-1.031913,...,-0.256128,-1.057681,-0.366751,-0.213519,0.135477,-1.109008,-1.247979,-1.692318,1,train
4,renato sobral,david heath,2007-08-25,0.662406,2.022037,0.780985,-0.227361,0.343272,-1.177648,-1.712630,...,-0.532528,-0.604273,-0.366751,-0.758923,0.135477,0.583803,0.477503,0.899094,1,train


In [7]:
red_id_col = 'id_red'
blue_id_col = 'id_blue'
date_col = 'date'

df_graph = build_train_test(df_v2, all_cols)
data = build_temporal_graph(df_graph, red_id_col, blue_id_col, date_col, model_cols_red, model_cols_blue)


In [8]:
data

Data(x=[1713, 40], edge_index=[2, 10008], y=[10008, 1], train_mask=[10008], test_mask=[10008])

In [9]:
(data.train_mask.sum(), data.test_mask.sum())

(tensor(8506), tensor(1502))

In [10]:
import torch
import torch.nn as nn
from torch_geometric.nn import GATConv, global_mean_pool


class EdgeClassifier(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes=2):
        super().__init__()
        
        # GAT layers
        self.conv1 = GATConv(in_channels, hidden_channels, heads=2, concat=True)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels * 2)
        
        self.conv2 = GATConv(hidden_channels*2, hidden_channels, heads=1, concat=True)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        
        self.conv3 = GATConv(hidden_channels, hidden_channels, heads=1, concat=True)
        self.bn3 = torch.nn.BatchNorm1d(hidden_channels)
        
        self.dropout = torch.nn.Dropout(p=0.3)
        
        # Edge MLP
        self.edge_mlp = torch.nn.Sequential(
            torch.nn.Linear(4*hidden_channels, hidden_channels*2),  # node embeddings + graph pooled
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.3),
            torch.nn.Linear(hidden_channels*2, hidden_channels),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.3),
            torch.nn.Linear(hidden_channels, num_classes)
        )

    def forward(self, x, edge_index, batch):
        # Node embeddings through 3-layer GAT
        x1 = F.relu(self.bn1(self.conv1(x, edge_index)))
        x1 = self.dropout(x1)
        x2 = F.relu(self.bn2(self.conv2(x1, edge_index)))
        x2 = self.dropout(x2)
        x = F.relu(self.bn3(self.conv3(x2, edge_index)))
        x = self.dropout(x)
        
        batch = torch.zeros(x.size(0), dtype=torch.long, device=x.device)
        graph_feat = global_mean_pool(x, batch)      # [1, hidden_channels]
        graph_feat_expanded = graph_feat[batch]      # [num_nodes, hidden_channel
        # Graph-level pooling
        graph_feat = global_mean_pool(x, batch)  # [num_graphs, hidden_channels]
        # Map graph-level feature to nodes
        graph_feat_expanded = graph_feat[batch]  # repeat per node

        # Gather embeddings for source and target of each edge
        src, dst = edge_index
        edge_feat = torch.cat([
            x[src],                # [num_edges, hidden_channels]
            x[dst],                # [num_edges, hidden_channels]
            graph_feat_expanded[src],  # [num_edges, hidden_channels]
            graph_feat_expanded[dst]   # [num_edges, hidden_channels]
        ], dim=1)  # total: 4 * hidden_channels

        # Predict edge outcomes
        return self.edge_mlp(edge_feat)

# ----------------------------
node_feat_dim = len(model_cols_red)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EdgeClassifier(node_feat_dim, hidden_channels=32).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=5e-4)

for epoch in range(500):
    model.train()
    optimizer.zero_grad()
   
    # Forward pass
    out = model(data.x, data.edge_index, data.batch)  # no edge_attr needed

    # Compute loss only on train edges
    train_out = out[data.train_mask]
    train_y = data.y[data.train_mask].view(-1).long()  # cross_entropy expects long
    loss = F.cross_entropy(train_out, train_y)

    loss.backward()
    optimizer.step()

    # Evaluate on test edges
    model.eval()
    with torch.no_grad():
        test_out = out[data.test_mask]
        test_y = data.y[data.test_mask].view(-1).long()
        pred = test_out.argmax(dim=1)
        acc = (pred == test_y).float().mean().item()

    if epoch % 5 == 0:
        print(f"Epoch {epoch}, Loss {loss.item():.4f}, Test Acc {acc:.4f}")

Epoch 0, Loss 0.6950, Test Acc 0.4920
Epoch 5, Loss 0.6921, Test Acc 0.5193
Epoch 10, Loss 0.6892, Test Acc 0.4987
Epoch 15, Loss 0.6874, Test Acc 0.5047
Epoch 20, Loss 0.6810, Test Acc 0.5087
Epoch 25, Loss 0.6755, Test Acc 0.5120
Epoch 30, Loss 0.6710, Test Acc 0.4927
Epoch 35, Loss 0.6561, Test Acc 0.5346
Epoch 40, Loss 0.6482, Test Acc 0.5439
Epoch 45, Loss 0.6406, Test Acc 0.5473
Epoch 50, Loss 0.6322, Test Acc 0.5426
Epoch 55, Loss 0.6216, Test Acc 0.5559
Epoch 60, Loss 0.6173, Test Acc 0.5399
Epoch 65, Loss 0.6030, Test Acc 0.5679
Epoch 70, Loss 0.5941, Test Acc 0.5706
Epoch 75, Loss 0.5916, Test Acc 0.5712
Epoch 80, Loss 0.5867, Test Acc 0.5772
Epoch 85, Loss 0.5791, Test Acc 0.5732
Epoch 90, Loss 0.5842, Test Acc 0.5726
Epoch 95, Loss 0.5693, Test Acc 0.6119
Epoch 100, Loss 0.5761, Test Acc 0.6012
Epoch 105, Loss 0.5713, Test Acc 0.6012
Epoch 110, Loss 0.5601, Test Acc 0.6072
Epoch 115, Loss 0.5615, Test Acc 0.6059
Epoch 120, Loss 0.5547, Test Acc 0.5972
Epoch 125, Loss 0.5521